<a href="https://colab.research.google.com/github/Vishy-A/MachineLearningIntro/blob/main/HW5/4105HW5P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
#Vishy Adusumilli
#801185663
#HW 5 Problems 2
import numpy as np
import pandas as pd
import seaborn as sns;
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems = 2, linewidth = 75)
import matplotlib.pyplot as plt
from google.colab import drive
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [130]:
varsnums = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']

def binarymap(inp):
  return inp.map({'yes' : 1, 'no' : 0, })

standardScaler = StandardScaler()

dataset[varsnums] = standardScaler.fit_transform(dataset[varsnums])
np.random.seed(0)
dataset.head()



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,yes,no,no,no,yes,1.517692,yes,furnished
1,4.004484,1.757010,1.403419,5.405809,2.532024,yes,no,no,no,yes,2.679409,no,furnished
2,4.004484,2.218232,0.047278,1.421812,0.224410,yes,no,yes,no,no,1.517692,yes,semi-furnished
3,3.985755,1.083624,1.403419,1.421812,0.224410,yes,no,yes,no,yes,2.679409,yes,furnished
4,3.554979,1.046726,1.403419,-0.570187,0.224410,yes,yes,yes,no,yes,1.517692,no,furnished


In [131]:
x = dataset[varsnums].values
y = dataset.pop('price')

x = torch.tensor(x)
y = torch.tensor(y)

In [132]:
samples = x.shape[0]
n_val = int(.2 * samples)

shuffledindices = torch.randperm(samples)

trainindices = shuffledindices[:-n_val]
valindices = shuffledindices[-n_val:]

trainindices, valindices



(tensor([389, 505, 396, 339,  78, 269, 323, 503,  65,   1,  20, 110,  75,
         210, 115, 120, 491,  16, 442, 431, 524, 453, 354, 382, 263,  33,
          99, 157, 493,  80,  86,  28, 291, 180, 519,  66, 533, 398,  22,
         534, 322, 506, 361, 358, 304, 494, 329,  57, 455,  21, 282, 131,
         521,   9, 297,  43, 245, 474, 315, 343,  88, 483, 470, 278,  49,
         130, 501, 441, 325, 122, 232, 407, 352, 307, 168, 517,  73, 509,
         541,  53, 527,  90, 428, 132, 153,  64,  40, 370, 488,  26, 369,
         385,  23, 152, 500, 499, 197, 309,  42, 465,   0, 133,  69, 409,
         512, 418, 539, 338, 514, 336, 164,  45,  11, 166, 394, 266, 101,
         106, 525, 342, 346, 202, 143, 218, 170, 192, 140, 114,  27, 270,
         502, 149, 363, 319, 265, 227, 532, 317, 139, 125,  47, 254, 151,
         460,  60, 305, 259, 310,  36, 415,  29, 456, 410, 107, 178, 340,
         234, 112, 296, 221, 290, 507, 182, 158, 495, 240, 177,  94, 348,
         468, 414, 353,  17, 448, 138,

In [133]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]

xntrain = .1 * xtrain
xnval = .1 * valx

In [134]:
def model(x, w1, w2, w3, w4, w5, b):
  return w1 * x[:,0] + w2 * x[:,1] + w3 * x[:,2] + w4 * x[:,3] + w5 * x[:,4] + b

In [135]:
def loss_func(t_p,t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [136]:
def training(epochs, optimizer, params, xtrain, ytrain, xtest, ytest):
  for epoch in range(1, epochs + 1):
    t_p = model(xtrain, *params)
    loss = loss_func(t_p, ytrain)

    v_p = model(xtest, *params)
    vloss = loss_func(v_p, ytest)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



    if epoch % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %g' % (epoch, float(loss), float(vloss)))


  return params

In [137]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .1
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.446124, Validation Loss 0.409245
Epoch 1000, Loss 0.446124, Validation Loss 0.409245
Epoch 1500, Loss 0.446124, Validation Loss 0.409245
Epoch 2000, Loss 0.446124, Validation Loss 0.409245
Epoch 2500, Loss 0.446124, Validation Loss 0.409245
Epoch 3000, Loss 0.446124, Validation Loss 0.409245
Epoch 3500, Loss 0.446124, Validation Loss 0.409245
Epoch 4000, Loss 0.446124, Validation Loss 0.409245
Epoch 4500, Loss 0.446124, Validation Loss 0.409245
Epoch 5000, Loss 0.446124, Validation Loss 0.409245


tensor([0.3856, 0.0754, 0.2973, 0.2576, 0.1746, 0.0148],
       requires_grad=True)

In [138]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .01
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.446124, Validation Loss 0.409265
Epoch 1000, Loss 0.446124, Validation Loss 0.409245
Epoch 1500, Loss 0.446124, Validation Loss 0.409245
Epoch 2000, Loss 0.446124, Validation Loss 0.409245
Epoch 2500, Loss 0.446124, Validation Loss 0.409245
Epoch 3000, Loss 0.446124, Validation Loss 0.409245
Epoch 3500, Loss 0.446124, Validation Loss 0.409245
Epoch 4000, Loss 0.446124, Validation Loss 0.409245
Epoch 4500, Loss 0.446124, Validation Loss 0.409245
Epoch 5000, Loss 0.446124, Validation Loss 0.409245


tensor([0.3856, 0.0754, 0.2973, 0.2576, 0.1746, 0.0148],
       requires_grad=True)

In [139]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .001
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.563762, Validation Loss 0.52086
Epoch 1000, Loss 0.450580, Validation Loss 0.417291
Epoch 1500, Loss 0.446794, Validation Loss 0.411563
Epoch 2000, Loss 0.446294, Validation Loss 0.410266
Epoch 2500, Loss 0.446170, Validation Loss 0.409752
Epoch 3000, Loss 0.446136, Validation Loss 0.409508
Epoch 3500, Loss 0.446127, Validation Loss 0.409383
Epoch 4000, Loss 0.446125, Validation Loss 0.409318
Epoch 4500, Loss 0.446124, Validation Loss 0.409284
Epoch 5000, Loss 0.446124, Validation Loss 0.409265


tensor([0.3855, 0.0756, 0.2971, 0.2576, 0.1747, 0.0147],
       requires_grad=True)

In [140]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .0001
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 4.363728, Validation Loss 3.66176
Epoch 1000, Loss 3.082972, Validation Loss 2.60944
Epoch 1500, Loss 2.222379, Validation Loss 1.90086
Epoch 2000, Loss 1.643898, Validation Loss 1.42327
Epoch 2500, Loss 1.254879, Validation Loss 1.10098
Epoch 3000, Loss 0.993118, Validation Loss 0.883162
Epoch 3500, Loss 0.816858, Validation Loss 0.735666
Epoch 4000, Loss 0.698064, Validation Loss 0.635555
Epoch 4500, Loss 0.617907, Validation Loss 0.567405
Epoch 5000, Loss 0.563743, Validation Loss 0.520844


tensor([0.4602, 0.2388, 0.3321, 0.3530, 0.3347, 0.0022],
       requires_grad=True)

In [141]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .1
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.446124, Validation Loss 0.409245
Epoch 1000, Loss 0.446124, Validation Loss 0.409245
Epoch 1500, Loss 0.446124, Validation Loss 0.409245
Epoch 2000, Loss 0.446124, Validation Loss 0.409245
Epoch 2500, Loss 0.446124, Validation Loss 0.4092
Epoch 3000, Loss 0.446124, Validation Loss 0.40925
Epoch 3500, Loss 0.446124, Validation Loss 0.409245
Epoch 4000, Loss 0.446180, Validation Loss 0.408192
Epoch 4500, Loss 0.446124, Validation Loss 0.409245
Epoch 5000, Loss 0.446124, Validation Loss 0.409245


tensor([0.3856, 0.0754, 0.2973, 0.2576, 0.1746, 0.0148],
       requires_grad=True)

In [142]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .01
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.446124, Validation Loss 0.409279
Epoch 1000, Loss 0.446124, Validation Loss 0.409245
Epoch 1500, Loss 0.446124, Validation Loss 0.409245
Epoch 2000, Loss 0.446124, Validation Loss 0.409245
Epoch 2500, Loss 0.446124, Validation Loss 0.409225
Epoch 3000, Loss 0.446124, Validation Loss 0.409245
Epoch 3500, Loss 0.446124, Validation Loss 0.409245
Epoch 4000, Loss 0.446124, Validation Loss 0.409245
Epoch 4500, Loss 0.446124, Validation Loss 0.409245
Epoch 5000, Loss 0.446124, Validation Loss 0.40924


tensor([0.3856, 0.0754, 0.2973, 0.2576, 0.1746, 0.0141],
       requires_grad=True)

In [143]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .001
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 1.672432, Validation Loss 1.44626
Epoch 1000, Loss 0.590123, Validation Loss 0.546972
Epoch 1500, Loss 0.460239, Validation Loss 0.429885
Epoch 2000, Loss 0.448598, Validation Loss 0.41493
Epoch 2500, Loss 0.446534, Validation Loss 0.411037
Epoch 3000, Loss 0.446168, Validation Loss 0.409772
Epoch 3500, Loss 0.446126, Validation Loss 0.409369
Epoch 4000, Loss 0.446124, Validation Loss 0.409265
Epoch 4500, Loss 0.446124, Validation Loss 0.409247
Epoch 5000, Loss 0.446124, Validation Loss 0.409245


tensor([0.3856, 0.0754, 0.2973, 0.2576, 0.1746, 0.0148],
       requires_grad=True)

In [144]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .0001
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 5.546566, Validation Loss 4.63297
Epoch 1000, Loss 4.892828, Validation Loss 4.09575
Epoch 1500, Loss 4.299613, Validation Loss 3.60818
Epoch 2000, Loss 3.761800, Validation Loss 3.16602
Epoch 2500, Loss 3.275229, Validation Loss 2.76587
Epoch 3000, Loss 2.836529, Validation Loss 2.40494
Epoch 3500, Loss 2.442949, Validation Loss 2.08099
Epoch 4000, Loss 2.092196, Validation Loss 1.79211
Epoch 4500, Loss 1.782292, Validation Loss 1.53669
Epoch 5000, Loss 1.511433, Validation Loss 1.31324


tensor([0.5649, 0.5519, 0.5588, 0.5569, 0.5561, 0.0033],
       requires_grad=True)